## Data fields & description:
- Project Number -- uid
- Project Title -- Name of the project. Mandatory field.
- Project Title English -- Name of the project in English. Optional field.- 
- Reponsible Applican -- Person who submitted the application to the SNSF. Mandatory field.
- Funding Instrument -- Funding scheme
- Funding Instrument Hierarchy -- Top Level Hierarchy Funding Instrument.
- Institution -- Research institution where the project will largely be carried out. Typically, this is the institution or specific lab where the responsible applicant works. "This field is not structured, but its content is consolidated internally." There is no link in the database to the field University.
- University -- This is the institution where the project will largely be carried out according to the application. Pick list. This field is only filled if the research is carried out at a Swiss institution, otherwise the field remains blank. In the case of mobility fellowships, it is generally left empty.
- Discipline Number -- Number of the discipline. Only the main discipline from a list of disciplines supplied by the SNSF.
- Discipline Name -- Name of the discipline.
- Discipline Name Hierarchy -- Top level of discipline.
- Start Date -- Date on which the project starts.
- End Date -- Actual end date of the project (updated if changed).
- Approved Amount -- Approved amount, changed if the award is corrected. This amount is not indicated in the case of mobility fellowships.
- Keywords -- unstructured field.
- Abstract (Project) -- The scientific abstract of the research project. The researchers are responsible for the contents.
- Lay Summary - Lead -- Summarise the most important information in the Lay Summary.
- Lay Summary -- Summary of a research project aimed at laypeople.

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML
import folium
import urllib
import requests
%matplotlib inline

In [2]:
df = pd.read_csv('./data/P3_GrantExport.csv', sep=";")

In [3]:
df.head()

,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN


The documentation of fields mentions that institutes from outside Switzerland don't have an entry in University. So it's safe to throw away all rows has 'nicht zuteilbar - NA' in university column.

In [4]:
df_Nicht = df[df['University'] == 'Nicht zuteilbar - NA']
df = df[df['University'] != 'Nicht zuteilbar - NA']

In [ ]:
with pd.option_context('display.max_rows', 999, 'display.max_columns', 3):
    print(df['University'].value_counts())

In [6]:
df_null = df[df['University'].isnull()]

In [7]:
list_of_universities = set(df['University'].unique())

In [29]:
GEONAMES_SEARCH_URL = 'http://api.geonames.org/searchJSON'

## GeoNames Search Webservice:
GeoNames api offers services in XML and JSON format. We use JSON format as it is easier to extract data from it.

In [64]:
def search(params):
    """ This function returns a json file containing the results from the
    Geonames website. It turns out that it can only recover 5/76 """
    url = GEONAMES_SEARCH_URL + '?' + urllib.parse.urlencode(params, encoding='utf8')

    try:
        response = requests.get(url)
    except:
        raise ServiceException(url)

    return response.json()

In [67]:
lst = []
for university in list_of_universities:
    response = search({'q': university, 'maxRows': 1, 'country': 'CH', 'username': 'sorooshafiee'})
    lst.append(response['totalResultsCount'])
print('Number of the recovered universities conton: {}/{}'.format(sum(x != 0 for x in lst), len(lst)))

Number of the recovered universities conton: 6/77


## Google API


In [14]:
from geopy.geocoders import Nominatim, Bing, ArcGIS, YahooPlaceFinder, IGNFrance, OpenMapQuest
geolocator = Nominatim()
location = geolocator.geocode("Universität St. Gallen - SG")
print(location)

Universität, Tannenstrasse, Rotmonten, St. Gallen, Wahlkreis St. Gallen, Sankt Gallen, 9010, Schweiz, Suisse, Svizzera, Svizra


In [9]:
import urllib
params = (q=name_arg, username=GEONAMES_USERNAME, isNameRequired='true')
urllib.urlencode(q=name_arg, username=GEONAMES_USERNAME, isNameRequired='true')

SyntaxError: invalid syntax (<ipython-input-9-8a690bc40b90>, line 2)

In [13]:
urllib.parse.urlencode({'q': 'epfl uni', 'isNameRequired': 'true'})

'q=epfl+uni&isNameRequired=true'

In [11]:
import urllib

In [26]:
def search(params):

    url = GEONAMES_SEARCH_URL + "?" + urllib.parse.urlencode.urlencode(params)

    try:
        response = urllib.request.urlopen(url)
    except:
        raise ServiceException(url)

    return json.load(response)

In [28]:
url = r'http://www.geonames.org/search.html?q=epfl&country=CH'

In [54]:
try:
    response = urllib.request.urlopen(url)
except:
    print('false')

In [74]:
a = json.load(response)

TypeError: the JSON object must be str, not 'bytes'

In [17]:
import requests

In [18]:
resp = requests.get(url)

In [19]:
resp

<Response [200]>

In [20]:
resp.json()

{'geonames': [], 'totalResultsCount': 0}

In [68]:
response.read()

b''

In [59]:
 r = requests.get('https://api.github.com/users/burhankhalid')

In [60]:
r

<Response [404]>

In [89]:
import json
import urllib

def get_geonames(lat, lng, types):
    url = 'http://maps.googleapis.com/maps/api/geocode/json' + \
            '?latlng={},{}&sensor=false'.format(lat, lng)
    jsondata = json.load(urllib.request.urlopen(url))
    address_comps = jsondata['results'][0]['address_components']
    filter_method = lambda x: len(set(x['types']).intersection(types))
    return filter(filter_method, address_comps)

lat, lng = 46.5, 6.6
types = ['locality', 'administrative_area_level_1']


In [90]:
url = 'http://maps.googleapis.com/maps/api/geocode/json' + \
            '?latlng={},{}&sensor=false'.format(lat, lng)

In [91]:
url

'http://maps.googleapis.com/maps/api/geocode/json?latlng=46.5,6.6&sensor=false'

In [80]:
data = requests.get(url)

In [81]:
data

<Response [200]>

In [88]:
data.json()['results'][0]['address_components']

[{'long_name': 'Tångtunneln', 'short_name': 'Tångtunneln', 'types': ['route']},
 {'long_name': 'Stockholm',
  'short_name': 'Stockholm',
  'types': ['locality', 'political']},
 {'long_name': 'Johanneshov',
  'short_name': 'Johanneshov',
  'types': ['postal_town']},
 {'long_name': 'Stockholms län',
  'short_name': 'Stockholms län',
  'types': ['administrative_area_level_1', 'political']},
 {'long_name': 'Sweden',
  'short_name': 'SE',
  'types': ['country', 'political']},
 {'long_name': '121 45', 'short_name': '121 45', 'types': ['postal_code']}]

In [69]:
!pip install simplejson

In [70]:
!head api_key

head: cannot open 'api_key' for reading: No such file or directory


In [74]:
with open('data/api_key.txt', 'r') as in_file:
    api_key = in_file.read()